### [사전 학습된 모델 테스트]
- `https://github.com/BM-K/KoSentenceBERT_SKT`
- 삼성전자와 LG화학 관련 뉴스URL 군집 각 2개
  - `samsung1`, `samsung2`, `lgchem1`, `lgchem2`
- 뉴스 타이틀의 유사성 분석 후 라벨링

### [이슈 및 세팅 시 필요한 내용 정리]
- 로컬에서 돌렸을 때 경로 충돌 및 다양한 문제 발생
  - [참고] `https://github.com/BM-K/KoSentenceBERT_SKT/issues/6`
  - 도커허브에 올려주신 이미지로 컨테이너 구성 후 테스트

- 컨테이너 내에서 git clone 불가
  - [참고] `https://github.com/moby/moby/issues/16600`
  - `/etc/docker/daemon.json` 생성 후 dns 세팅 `{"dns":["8.8.8.8"]}`

- 사전 학습된 PT 파일 복사 (로컬파일을 도커 컨테이너로 전송)
  - `docker cp <전송할 파일> <컨테이너명>:<저장할 파일경로>`
  - 역 : `docker cp <컨테이너명>:<전송할 파일> <호스트 경로>`

In [ ]:
# 사전 모아둔 URL 리스트
import urls

In [ ]:
# 불용어 사전 활용, 전처리에 활용
# https://www.ranks.nl/stopwords/korean

def get_stopwords():
    stopwords = ["특파원", "현지시간", "현지시각", "현지", "기자", "단독", "한편"]
    f = open("./korean_stopwords.tsv", "r", encoding="UTF-8")
    while True:
        line = f.readline()
        if not line:
            break
        stopwords.append(line.rstrip())
    return set(stopwords)

In [ ]:
# 불용어 + 기업이름(keywords) 삭제
def preprocessing(url_list, stopwords, keywords, dummy_title="각캭콕쿅 눈누날라"):
    # 불용어 + 기업이름 삭제 (전처리)
    from newspaper import Article

    titles = []
    for i, url in enumerate(url_list):
        try:
            article = Article(url, language="ko")
            article.download()
            article.parse()
            if not article.title or not article.text:
                raise (Exception)

            words = ""
            for word in article.title.split():
                is_included = True
                for keyword in keywords:
                    if keyword in word:  # 기업이름
                        is_included = False
                        break
                if not is_included:
                    continue
                if word not in stopwords:
                    words += word + " "
            titles.append(words.rstrip())

        except Exception:
            titles.append(dummy_title)
            print(f"{i}번 문서 dummy 값으로 처리")
            continue

    return titles

In [ ]:
# 코사인 유사도 기반 군집 라벨링
def get_cos_sim(corpus):
    from KoSentenceBERT_SKTBERT.sentence_transformers import SentenceTransformer, util
    import numpy as np

    model_path = "KoSentenceBERT_SKTBERT/output/training_sts"
    embedder = SentenceTransformer(model_path)

    embeddings = embedder.encode(corpus, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(embeddings, embeddings)

    labels = [i for i in range(len(corpus))]

    def find(x):
        if labels[x] != x:
            labels[x] = find(labels[x])
        return labels[x]

    for i in range(len(corpus)):
        for j in range(len(corpus)):
            if cos_scores[i, j] > 0.75:
                fi, fj = find(i), find(j)
                if fi < fj:
                    labels[fj] = fi
                elif fj < fi:
                    labels[fi] = fj
    for x in range(len(corpus)):
        find(x)
    return labels

In [ ]:
# 각 라벨별 개수 파악
def classifier(titles, labels, dummy_title="각캭콕쿅 눈누날라"):
    labels = set(labels)
    result = []
    for idx in labels:
        if titles[idx] == dummy_title:
            continue
        result.append(idx)
    return result

In [ ]:
if __name__ == "__main__":
    stopwords = get_stopwords()
    url_list = urls.lgchem1 + urls.lgchem2
    keywords = ["LG", "lg"]

    titles = preprocessing(url_list, stopwords, keywords)
    labels = get_cos_sim(titles)
    K = len(urls.lgchem1)
    
    print("\n[라벨링]")
    print(f"lgchem1 : {labels[:K]}")
    print(f"lgchem2 : {labels[K:]}")

    print("\n[분류 결과]")
    result = classifier(titles, labels)
    print(result)
    for idx in result:
        print(titles[idx])
